# Assignment 2: Forest Fires - Best Model

Forest fires dataset found here:
https://archive.ics.uci.edu/ml/datasets/forest+fires
Please place csv in route

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
from sklearn.svm import SVR

#### Here We fit the best performing features and parameters to our best performing model, the SVM

# Preprocessing

First we set a reproducible seed and separate out the target feature 

In [39]:
np.random.seed()

raw = pd.read_csv('forestfires.csv')

# subset X columns and Y column
X = raw.iloc[:, 0:12]
y = np.array(raw.iloc[:, 12])

We then perform the sine cosine feature transformation of the month and day features by first mapping them to sequential numerics and then generating the following features for each of them:
$$x_{\sin }=\sin \left(\frac{2 * \pi * x}{\max (x)}\right)$$

$$x_{\cos }=\cos \left(\frac{2 * \pi * x}{\max (x)}\right)$$

In [40]:


# cos sin transform categorical sequential features
# map features to numerics
monthDict = dict(zip(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],[1,2,3,4,5,6,7,8,9,10,11,12]))
dayDict = dict(zip(['mon','tue','wed','thu','fri','sat','sun'],[1,2,3,4,5,6,7]))

X['month'] = X['month'].map(monthDict).astype(float)
X['day'] = X['day'].map(dayDict).astype(float)


# map each cyclical variable onto a circle so lowest value for that variable appears right next to the largest value.
X['day_sin'] = np.sin(X.day*(2.*np.pi/7))
X['day_cos'] = np.cos(X.day*(2.*np.pi/7))
X['mnth_sin'] = np.sin(X.month*(2.*np.pi/12))
X['mnth_cos'] = np.cos(X.month*(2.*np.pi/12))

Next we drop the original month and day features and convert the ints to floats

In [41]:
# drop original categorical variables
X = X.drop(['month', 'day'], 1)

# fix int warning
X['X'] = X['X'].astype(float)
X['Y'] = X['Y'].astype(float)
X['RH'] = X['RH'].astype(float)

# Modelling

We then define the negative log likelihood function as follows:
$$N N L=-\log p\left(y_{*} | D, x_{*}\right)=\frac{1}{2} \log \left(2 \pi \sigma_{*}^{2}\right)+\frac{\left(y_{*}-\bar{f}\left(x_{*}\right)\right)^{2}}{2 \sigma_{*}^{2}}$$

In [42]:
# define negative log likelihood of sample
def negative_log_likelihood(y, p):
    result = 0.5 * np.log(2 * np.pi * np.var(p)) + (((y - np.mean(p)) ** 2) / (2 * np.var(p)))
    return result


The feature sets to tune over in the inner fold are then defined. The best model utilised STFWIM in 8 out of 10 of its outer folds, so we only allow for selecting this parameter here

In [43]:
#feature selection list
#for final result only use one 
STFWIM = ['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain',
       'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']


In this step we define the nested cross validation function. 

This is elaborated on in the "other models" notebook. Here we only provide the best parameters and feature set so the GridSearchCV is only utilised to provide ease of scoring.

In [44]:


def nested_crossval(model , parameters):
    # build pipeline
    pipeline = Pipeline([('scaler', MinMaxScaler()),
                         ('estimator',
                          TransformedTargetRegressor(regressor=model
                                                     , func=np.log1p, inverse_func=np.expm1))])

    # define outer and inner folds
    outer_kv = KFold(n_splits=10, shuffle=True, random_state=42)
    inner_kv = KFold(n_splits=10, shuffle=True, random_state=42)
    
    #instantiate inner CV grid search 
    cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=inner_kv,
                      scoring="neg_mean_squared_error", n_jobs=-1, verbose=False)
    
    #create list of feature spaces to search and list of result to persist
    feature_space = [STFWIM]
    inner_result = []
    outer_result = []
    saving_inner_results = []
    i = 0
    for train, test in outer_kv.split(X):
        print("run", i)
        # loop over feature space using only training data and cross validator hyper param tuner
        for f in feature_space: 
            cv.fit(X.loc[X.index[train], f], y[train])
            # persist models to fit best on training set
            inner_result.append([cv, cv.best_params_, f, cv.best_score_])
            #print(f)
            #print(cv.best_score_)
        # persits and reset inner result for next fold
        inner_df = pd.DataFrame(inner_result)
        # reset inner result
        inner_result = []
        # receive best model of run to fit on test set
        best_params_arg = inner_df.loc[:, 3].argmax()
        best_params = inner_df.iloc[best_params_arg, :]
        # fit best cv model hyper parameters on best feature set for that fold
        bcv = best_params[0]
        bfs = best_params[2]
        bcv.fit(X.loc[X.index[train], bfs], y[train])

        # get training/val and test scores
        train_score = best_params[3]
        test_score = bcv.score(X.loc[X.index[test], bfs], y[test])

        # get predictions and retrieve nll of test folds
        y_preds = cv.predict(X.loc[X.index[test], bfs])
        mae = mean_absolute_error(y[test], y_preds)
        nllval = negative_log_likelihood(y[test], y_preds)
        mean_nll = np.mean(nllval)

        outer_result.append([i, train_score, test_score, mae, bfs, best_params[1], y[test], nllval, mean_nll])
        i += 1

    testing = pd.DataFrame(outer_result)
    testing.columns = ['fold_number', 'train_nmse', 'test_nmse', 'test_mae', 'best_feature_set', 'best_hyperparams',
                       'test_set', 'nll', 'mean_nll']

    return testing


# Best Model Results

For the best overall performance we submit the SVM. It has close performance to both the 'MAD' and RMSE benchmark (even though we evalute on unseen data), and also a more competitive NLL when compared to the LR. 

9 out of 10 outer folds utilised a linear kernel and 7 out of 10 outer folds set C to 1000, so we use these hyperparameters here.

In [45]:
start=datetime.now()
svr_results = nested_crossval(SVR(),
                              {'estimator__regressor__C': [1000],
                               'estimator__regressor__kernel': ['linear']})

svmruntime = datetime.now()-start


run 0


C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Depre

run 1


C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Depre

run 2


...

C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Depre

run 8


C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Depre

run 9


C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Depre

The average out of sample results for the 10 fits are reported below:
This number will differ from the 10 model averaged scores reported as it doesnt allow for model averaging over all ten folds ouput

In [46]:
#printing results
print("average SVM test fold RMSE (Benchmark: 63.7)", np.sqrt(np.mean(np.negative(svr_results['test_nmse']))))
print("average SVM test fold MSE:",np.mean(np.negative(svr_results['test_nmse'])))
print("average SVM test fold MAE/'MAD' (Benchmark:12.71):", np.mean(svr_results['test_mae']))
print("average SVM test fold NLL:", np.mean(svr_results['mean_nll']))
print("SVM Fitting Runtime:" ,svmruntime)

average SVM test fold RMSE (Benchmark: 63.7) 64.65524244315229
average SVM test fold MSE: 4180.300375382801
average SVM test fold MAE/'MAD' (Benchmark:12.71): 12.874752378055806
average SVM test fold NLL: 3917.199414938219
SVM Fitting Runtime: 0:00:17.975034
